<a href="https://colab.research.google.com/github/Jyothirmai987/project_assignment/blob/main/cnn_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from google.colab import files
from tensorflow.keras.models import load_model
import json
from types import SimpleNamespace


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
class CNN:
    def pre_processing(self):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = keras.datasets.mnist.load_data()

        # normalize it to 0-1
        self.x_train = self.x_train / 255
        self.x_test = self.x_test / 255
        # converting 2d to 1d
        # Assuming grayscale images
        self.x_train = self.x_train.reshape((self.x_train.shape[0], 28, 28, 1))
        self.x_test = self.x_test.reshape((self.x_test.shape[0], 28, 28, 1))

    def cnn_model(self):
        self.model = models.Sequential([
            layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
    def load_config(self,path):
      with open(path,'r') as f:
        data=json.load(f)
      self.config_obj=SimpleNamespace(**data)




    def training(self):
        self.model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=self.config_obj.path,
                save_weights_only=True,
                monitor='val_accuracy',
                mode='max',
                save_best_only=True)
        self.model.compile(optimizer=self.config_obj.optimizer, loss=self.config_obj.loss, metrics=self.config_obj.metrics)
        self.model.fit(self.x_train, self.y_train, epochs=int(self.config_obj.epochs), validation_data=(self.x_test, self.y_test),callbacks=[self.model_checkpoint_callback])
        self.model.evaluate(self.x_test, self.y_test)

        # Save the trained model
        self.model.save(self.config_obj.model)
obj = CNN()
obj.pre_processing()
obj.cnn_model()
obj.load_config('/content/drive/MyDrive/project_assignment/config.json')
obj.training()


313/313 [==============================] - 3s 9ms/step - loss: 0.0840 - accuracy: 0.9746


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:


class test_model:
    def __init__(self):
        self.model = None

    def load_saved_model(self, model_path):
        # Load the saved model
        self.model = load_model(model_path)

    def test_img(self, image_path):
        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(28, 28), color_mode='grayscale')
        img_array = img_to_array(img)
        img_array /= 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict using the loaded model
        y_pred = self.model.predict(img_array)

        # Get the predicted label
        predicted_label = np.argmax(y_pred)

        return predicted_label
obj = test_model()
obj.load_saved_model("/content/mnist_cnn_model.h5")
predicted_label = obj.test_img('/content/drive/MyDrive/project_assignment/digit.jpg')
print("Predicted Label:", predicted_label)


1/1 [==============================] - 0s 168ms/step
Predicted Label: 5


# New Section

In [ ]:
with open('/content/config.json','r') as f:
  data=json.load(f)
config_obj=SimpleNamespace(**data)

In [ ]:
config_obj.model

'mnist_cnn_model.h5'